<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/Exp_13_LightGBM_Improvement_2(Stacking).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow boto3 awscli optuna lightgbm imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/8

Initially the plan was to use 3 base learners: XGBoost, LightGBM and SVM. And for the meta learner model, Logistic Regression. But that was taking lot of time to train. So the plan is to use LightGBM, Naive Bayes, and LogReg as base learners and KNN as meta learner.

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': '421MM4HJJ9KYQBZ5', 'HostId': '/BWUsr7gzBuUe9fkoeG2h5g8tJKSYqhNv01Kqd030MVaVZDzG0XE5M/KjMljlxp+vClepsdPQzleM7+I6KGiF+Gw1sDJ/n1b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '/BWUsr7gzBuUe9fkoeG2h5g8tJKSYqhNv01Kqd030MVaVZDzG0XE5M/KjMljlxp+vClepsdPQzleM7+I6KGiF+Gw1sDJ/n1b', 'x-amz-request-id': '421MM4HJJ9KYQBZ5', 'date': 'Mon, 20 Oct 2025 09:32:05 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [4]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [5]:
mlflow.set_experiment("exp 6 -LightGBM with HP Tuning")


<Experiment: artifact_location='s3://project1-mlflow-bucket/131227091685296502', creation_time=1760614823705, experiment_id='131227091685296502', last_update_time=1760614823705, lifecycle_stage='active', name='exp 6 -LightGBM with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB


In [7]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())
with mlflow.start_run() as run:
    print("Artifact URI:", mlflow.get_artifact_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/
Artifact URI: s3://project1-mlflow-bucket/131227091685296502/d7d5848bf50749ae8f69885e55ea3598/artifacts
🏃 View run nosy-crow-422 at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502/runs/d7d5848bf50749ae8f69885e55ea3598
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502


In [8]:
url = 'https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv'
df = pd.read_csv(url).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [15]:
nb_optuna_best = dict(alpha=0.0023)

lgb_optuna_best  = dict(objective='multiclass',
                        num_class=3,
                        metric="multi_logloss",
                        reg_alpha=0.1,  # L1
                        reg_lambda=0.1,  # L2
                        learning_rate=0.08081298097796712,
                        n_estimators=367,
                        max_depth=20,
                        n_jobs=1)

logreg_optuna_best = dict(max_iter=1000,
                          solver='lbfgs',
                          multi_class='multinomial')

from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, RandomOverSampler
import lightgbm as lgb

nb_clf = Pipeline([
    ("ros", RandomOverSampler(random_state=42)),
    ("nb", MultinomialNB(**nb_optuna_best))
])

lgb_clf = lgb.LGBMClassifier(**lgb_optuna_best, class_weight='balanced')

logreg_clf = LogisticRegression(**logreg_optuna_best, class_weight='balanced')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

stacking_model = StackingClassifier(
    estimators=[
        ("nb", nb_clf),
        ("lgb", lgb_clf),
        ("LogReg", logreg_clf)
    ],
    final_estimator=LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs', multi_class='multinomial'),
    stack_method="predict_proba",
    cv=cv,
    n_jobs=-1)

stacking_model.fit(X_train_tfidf, y_train)




/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


StackingClassifier(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimators=[('nb',
                                Pipeline(steps=[('ros',
                                                 RandomOverSampler(random_state=42)),
                                                ('nb',
                                                 MultinomialNB(alpha=0.0023))])),
                               ('lgb',
                                LGBMClassifier(class_weight='balanced',
                                               learning_rate=0.08081298097796712,
                                               max_depth=20,
                                               metric='multi_logloss',
                                               n_estimators=367, n_jobs=1,
                                               num_class=3,
                                               objective='multiclass',
                                               reg_alpha=0.1, reg_lambda=0.1)),
                               ('LogReg',
                                LogisticRegression(class_weight='balanced',
                                                   max_iter=1000,
                                                   multi_class='multinomial'))],
                   final_estimator=LogisticRegression(max_iter=1000,
                                                      multi_class='multinomial',
                                                      random_state=42),
                   n_jobs=-1, stack_method='predict_proba')

In [17]:
from sklearn.metrics import accuracy_score, classification_report

with mlflow.start_run():

        mlflow.set_tag("mlflow.runName", f"Stacking_model_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", "Stacking_model_3_BaseLearners")

        y_pred = stacking_model.predict(X_test_tfidf)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(stacking_model, "stacking_model")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/10/20 10:21:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/20 10:22:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Stacking_model_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502/runs/24e4cb4ab80e4617932c0aa69de29e8f
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502


In [16]:
y_pred = stacking_model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

          -1       0.82      0.75      0.78      1650
           0       0.86      0.95      0.91      2529
           1       0.90      0.87      0.88      3154

    accuracy                           0.87      7333
   macro avg       0.86      0.86      0.86      7333
weighted avg       0.87      0.87      0.87      7333



In [18]:

# Experimenting with 2 base learners

# Base learners

lightgbm_model_2 = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    class_weight="balanced",
    reg_alpha=0.1,  # L1
    reg_lambda=0.1,  # L2 ,
    learning_rate=0.08081298097796712,
    n_estimators=367,
    max_depth=20
)

logreg_model_2 = LogisticRegression(max_iter=1000, class_weight='balanced', solver='lbfgs', multi_class='multinomial')

# Meta-learner

knn_meta_learner = KNeighborsClassifier(n_neighbors=5)

stacking_model_2 = StackingClassifier(
    estimators=[
        ('lightgbm', lightgbm_model_2),
        ('logistic_regression', logreg_model_2)
    ],
    final_estimator=knn_meta_learner,
    cv=5)

stacking_model_2.fit(X_train_tfidf, y_train)

y_pred_2 = stacking_model_2.predict(X_test_tfidf)

print(classification_report(y_test, y_pred_2))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.069513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 131942
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 4434
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.189751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 103631
[LightGBM] [Info] Number of data points in the train set: 23463, number of used features: 3618
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.550568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 103361
[LightGBM] [Info] Number of data points in the train set: 23463, number of used features: 3617
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.285399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 104122
[LightGBM] [Info] Number of data points in the train set: 23463, number of used features: 3618
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.135976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 103504
[LightGBM] [Info] Number of data points in the train set: 23464, number of used features: 3602
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default v

              precision    recall  f1-score   support

          -1       0.79      0.76      0.77      1650
           0       0.87      0.93      0.90      2529
           1       0.89      0.86      0.87      3154

    accuracy                           0.86      7333
   macro avg       0.85      0.85      0.85      7333
weighted avg       0.86      0.86      0.86      7333

